# Lab Report 3: Predictive Regression

**Ethan Wang, Kevin Yang**  
**RSM338**  
**February 23, 2026**   

TEST

In [1]:
# Import Packages
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

pd.set_option('display.max_columns', None)
sns.set_theme(style='whitegrid')

## 1. Data Preparation

### 1.1 Load the Data, Parse Dates

First, we start by loading in "PredictorData2024", which contains data from the Welch and Goyal paper. We load the data in a dataframe and parse the date column to convert it to datetime format.

In [2]:
# Loading in the data
df = pd.read_excel("PredictorData2024.xlsx", sheet_name="Monthly")

# Parsing the date column
df['Date'] = pd.to_datetime(df['yyyymm'], format='%Y%m')

df.set_index('Date', inplace=True)

df.head()

c:\Users\kevin\Python Projects\RSM338\venv\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,yyyymm,Index,D12,E12,b/m,tbl,AAA,BAA,lty,ntis,Rfree,infl,ltr,corpr,svar,csp,CRSP_SPvw,CRSP_SPvwx
Date,,,,,,,,,,,,,,,,,,
1871-01-01,187101,4.44,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1871-02-01,187102,4.50,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004967,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1871-03-01,187103,4.61,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004525,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1871-04-01,187104,4.74,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004252,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1871-05-01,187105,4.86,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004643,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 1.2 Construct Derived Variables, Lag Inflation

We then proceed to generate derived variables that are necessary for our analysis. We create variables for excess market return, Log dividend-price ratio, Log dividend yield, Log earnings-price ratio, Log dividend-earnings ratio, term spread, default return spread, and default yield spread.

In addition, we add an additional one month lag for inflation, so that the inflation value used to predict month t returns is from month t-2. We do this by changing the 'infl' variable.

In [3]:
# Generating derived variables

# 1. Excess market return
df['ExRet'] = df['CRSP_SPvw'] - df['Rfree']

# 2. Log dividend-price ratio (d/p)
df['d_p'] = np.log(df['D12']) - np.log(df['Index'])

# 3. Log dividend yield (d/y) - Note the Index is lagged by 1 month
df['d_y'] = np.log(df['D12']) - np.log(df['Index'].shift(1))

# 4. Log earnings-price ratio (e/p)
df['e_p'] = np.log(df['E12']) - np.log(df['Index'])

# 5. Log dividend-earnings ratio (d/e)
df['d_e'] = np.log(df['D12']) - np.log(df['E12'])

# 6. Term spread (tms)
df['tms'] = df['lty'] - df['tbl']

# 7. Default return spread (dfr)
df['dfr'] = df['corpr'] - df['ltr']

# 8. Default yield spread (dfy)
df['dfy'] = df['BAA'] - df['AAA']

# 9. Additional Inflation Lag (Instruction 4d: lag it one extra month)
df['infl_lagged'] = df['infl'].shift(1)

df.head()

,yyyymm,Index,D12,E12,b/m,tbl,AAA,BAA,lty,ntis,Rfree,infl,ltr,corpr,svar,csp,CRSP_SPvw,CRSP_SPvwx,ExRet,d_p,d_y,e_p,d_e,tms,dfr,dfy,infl_lagged
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,
1871-01-01,187101,4.44,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.837728,NaN,-2.406945,-0.430783,NaN,NaN,NaN,NaN
1871-02-01,187102,4.50,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.851151,-2.837728,-2.420368,-0.430783,NaN,NaN,NaN,NaN
1871-03-01,187103,4.61,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.875302,-2.851151,-2.444519,-0.430783,NaN,NaN,NaN,NaN
1871-04-01,187104,4.74,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.903111,-2.875302,-2.472328,-0.430783,NaN,NaN,NaN,NaN
1871-05-01,187105,4.86,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004643,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.928112,-2.903111,-2.497329,-0.430783,NaN,NaN,NaN,NaN


### 1.3 Filter Sample

Next, we filter our dataframe to only keep the relevant data from December 1926 and onward. This is due to large amounts of missing data prior to then.

In [4]:
# Filter data to keep only rows from December 1926 onwards
df_filtered = df[df.index >= '1926-12-01']
df_filtered.drop(columns=['yyyymm'], inplace=True) # Drop redundant date column
df_filtered.head()

C:\Users\kevin\AppData\Local\Temp\ipykernel_27056\2633255666.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.drop(columns=['yyyymm'], inplace=True) # Drop redundant date column


,Index,D12,E12,b/m,tbl,AAA,BAA,lty,ntis,Rfree,infl,ltr,corpr,svar,csp,CRSP_SPvw,CRSP_SPvwx,ExRet,d_p,d_y,e_p,d_e,tms,dfr,dfy,infl_lagged
Date,,,,,,,,,,,,,,,,,,,,,,,,,,
1926-12-01,13.49,0.6900,1.240,0.441476,0.0307,0.0468,0.0568,0.0354,0.050876,0.0028,0.000000,0.0078,0.0056,0.000465,NaN,0.026047,0.020321,0.023247,-2.973012,-2.956570,-2.386837,-0.586175,0.0047,-0.0022,0.0100,0.005682
1927-01-01,13.21,0.6967,1.229,0.443706,0.0323,0.0466,0.0561,0.0351,0.050824,0.0025,-0.011299,0.0075,0.0056,0.000470,NaN,-0.002910,-0.005579,-0.005410,-2.942374,-2.963349,-2.374773,-0.567601,0.0028,-0.0019,0.0095,0.000000
1927-02-01,13.84,0.7033,1.218,0.428501,0.0329,0.0467,0.0559,0.0347,0.051668,0.0026,-0.005714,0.0088,0.0069,0.000287,NaN,0.045522,0.040566,0.042922,-2.979535,-2.932946,-2.430353,-0.549182,0.0018,-0.0019,0.0092,-0.011299
1927-03-01,13.93,0.7100,1.208,0.469765,0.0320,0.0462,0.0554,0.0331,0.046357,0.0030,-0.005747,0.0253,0.0083,0.000924,NaN,0.007324,0.002610,0.004324,-2.976535,-2.970053,-2.445079,-0.531456,0.0011,-0.0170,0.0092,-0.005714
1927-04-01,14.17,0.7167,1.197,0.456754,0.0339,0.0458,0.0548,0.0333,0.050514,0.0025,0.000000,-0.0005,0.0055,0.000603,NaN,0.013021,0.010907,0.010521,-2.984225,-2.967143,-2.471309,-0.512916,-0.0006,0.0060,0.0090,-0.005747


### 1.4 Verification

To verify that our data preparation has been completed correctly, we perform a test to validate the values of d/y, ExRet, and tms in January 1950. They should match the values -2.68, 0.0188, and 0.0108 respectively.

In [5]:
# Additional verification checkpoint
df_filtered.loc['1950-01-01', ['d_y', 'ExRet', 'tms']]

d_y     -2.679233
ExRet    0.018803
tms      0.010800
Name: 1950-01-01 00:00:00, dtype: float64

After verifying the data in the row for January 1950, we are confident that the data has been prepared correctly. We can now confidently proceed to our data analysis section.
<div style="page-break-after: always;"></div>

## 2 OLS Predictive Regressions
In this section, we will attempt to fit the predictive regression below.
$$
r_t = \alpha + \beta x_{t-1} + \varepsilon_t,\quad t = 1927/1,\ldots,2024/12.
$$
$r_t$ is the excess return of the value-weighted market portfolio and $x_{t−1}$ is a lagged predictive variable.

### 2.1 In-Sample Fit
Using the regression formula below, we will now run simple OLS regressions on the entire sample for each of the 14 predictors.

$$R_{IS}^{2}=1-\frac{\sum_{t=1}^{T}(r_{t}-\hat{r}_{t})^{2}}{\sum_{t=1}^{T}(r_{t}-\overline{r})^{2}}$$

In [6]:
# Define the predictors
predictors = [
    'd_e', 'svar', 'dfr', 'lty', 'ltr', 'infl_lagged', 
    'tms', 'tbl', 'dfy', 'd_p', 'd_y', 'e_p', 'b/m', 'ntis'
]

# Dictionary to store results
rsquared_results = {}

# Shift predictors by 1 to align x_{t-1} with r_t
df_lagged = df_filtered[predictors].shift(1)

for var in predictors:
    y = df_filtered['ExRet']
    X = sm.add_constant(df_lagged[var])

    valid_data = pd.concat([y, X], axis=1).dropna()
    y_clean = valid_data['ExRet']
    X_clean = valid_data.drop(columns=['ExRet'])

    model = sm.OLS(y_clean, X_clean).fit()
    rsquared_results[var] = model.rsquared

r2_table = pd.DataFrame.from_dict(rsquared_results, orient='index', columns=['In-Sample R2'])
r2_table.index.name = 'Predictor'
r2_table.sort_values(by='In-Sample R2', ascending=False)

,In-Sample R2
Predictor,
b/m,0.005524
ntis,0.004798
d_y,0.003137
tbl,0.003018
e_p,0.002724
dfy,0.002435
d_p,0.002377
lty,0.001995
ltr,0.001854


After fitting the OLS model, the R-squared results for all 14 variables are displayed in the table above. 

We notice that the R-squared values are all very small and near 0. Even the largest one, b/m, only explains 0.005 or 0.5% of the return variation. All of the time-lagged predictors can barely explain any changes in the monthly excess market return. Overall, the in-sample results appear very weak.

### 2.2 Out-of-Sample Evaluation
We will now use out-of-sample evaluation to evaluate how well our model can predict using untrained data.
Using the formula below, we can calculate the R-squared for our growing sample for each month 200 months from our starting period $(t_{201})$
$$R_{OOS}^{2}=1-\frac{\sum_{t\in OOS}(r_{t}-\hat{r}_{t})^{2}}{\sum_{t\in OOS}(r_{t}-\overline{r}_{t})^{2}}$$
This shows how much of the data in our sample is explained by our model, which theoretically should improve as the model is accesses more data.

However, simply predicting data is not particularly helpful. To see if it actually makes the investor money, we can use the Certainty Equivalent Value, described by the formula below:
$$\Delta CEV = \left( \left( \overline{r}_{p, \text{model}} - \frac{\gamma}{2}\sigma_{p, \text{model}}^{2} \right) - \left( \overline{r}_{p, \text{bench}} - \frac{\gamma}{2}\sigma_{p, \text{bench}}^{2} \right) \right) \times 12 \times 100$$
$r_p$ is the portfolio return over the out-of-sample period, $\gamma$ is the investor’s risk aversion parameter, and the variance term penalizes volatility. The factor of 12 annualizes the monthly difference, and multiplying by 100 expresses it in percentage points.

In [7]:
# Parameters
initial_window = 200
gamma = 2.5

# Add empty columns to table (if not already there)
r2_table['OOS R2'] = np.nan
r2_table['Delta CEV'] = np.nan

for var in predictors:
    
    sse_model = 0.0
    sse_bench = 0.0
    model_portfolio_rets = []
    bench_portfolio_rets = []
    
    # Create lagged predictor once
    x_lag = df_filtered[var].shift(1)
    y = df_filtered['ExRet']
    
    for t in range(initial_window, len(df_filtered)):
        
        # Expanding window: data up to t-1
        y_train = y.iloc[:t]
        X_train = sm.add_constant(x_lag.iloc[:t])
        
        train = pd.concat([y_train, X_train], axis=1).dropna()
        y_tr = train['ExRet']
        X_tr = train.drop(columns=['ExRet'])
        
        model = sm.OLS(y_tr, X_tr).fit()
        
        # Forecast r_t using x_{t-1}
        x_prev = x_lag.iloc[t]
        forecast = model.predict([1, x_prev])[0]
        
        bench_forecast = y_tr.mean()
        realized = y.iloc[t]
        
        # OOS R² components
        sse_model += (realized - forecast) ** 2
        sse_bench += (realized - bench_forecast) ** 2
        
        # Portfolio weights
        sigma_sq = y_tr.var(ddof=1)
        w_model = np.clip((1/gamma) * (forecast / sigma_sq), 0, 1.5)
        w_bench = np.clip((1/gamma) * (bench_forecast / sigma_sq), 0, 1.5)
        
        model_portfolio_rets.append(w_model * realized)
        bench_portfolio_rets.append(w_bench * realized)
    
    # Final metrics
    oos_r2 = 1 - (sse_model / sse_bench)
    
    ce_model = np.mean(model_portfolio_rets) - (gamma/2) * np.var(model_portfolio_rets, ddof=1)
    ce_bench = np.mean(bench_portfolio_rets) - (gamma/2) * np.var(bench_portfolio_rets, ddof=1)
    delta_cev = (ce_model - ce_bench) * 12 * 100
    
    r2_table.loc[var, 'OOS R2'] = oos_r2
    r2_table.loc[var, 'Delta CEV'] = delta_cev

# Format nicely
r2_table_display = r2_table.copy()
r2_table_display['In-Sample R2'] = r2_table_display['In-Sample R2'].round(4)
r2_table_display['OOS R2'] = r2_table_display['OOS R2'].round(4)
r2_table_display['Delta CEV'] = r2_table_display['Delta CEV'].round(2).astype(str) + '%'

display(r2_table_display.sort_values(by='OOS R2', ascending=False))

,In-Sample R2,OOS R2,Delta CEV
Predictor,,,
tbl,0.0030,0.0021,-0.05%
infl_lagged,0.0014,0.0005,-0.02%
tms,0.0010,-0.0000,0.59%
lty,0.0020,-0.0026,-0.1%
dfr,0.0006,-0.0037,-0.01%
d_p,0.0024,-0.0039,-1.44%
ntis,0.0048,-0.0047,0.97%
d_e,0.0000,-0.0053,0.18%
ltr,0.0019,-0.0053,0.03%


After performing the OOS evaluation, we can compare our R-squared and CEV values with the in-sample ones. For the in-sample predictors, the most promising ones were b/m book to market with an R-squared of 0.0055, and ntis net equity issuance with 0.0048. However, in the out-of-sample tests, b/m becomes the worst performing predictor with an r-squared of -0.0199 while ntis also fails with -0.0047. This shows that the model fails when evaluated on out-of-sample data, and the model has weak predictive power. The fall in R-squared can be seen as a sign of possible overfitting on the in-sample data.

### 2.3 Multivariate Regression
Now, we will use all the predictors simultaneously when regressing, but dropping $d/e$ and $tms$ to avoid multicollinearity, since they are linear combinations of other predictors.

In [8]:
# --- OOS Multivariate OLS
initial_window = 200
gamma = 2.5

# 12 predictors (drop d/e and tms as required)
multi_predictors = [
    'svar', 'dfr', 'lty', 'ltr', 'infl_lagged',
    'tbl', 'dfy', 'd_p', 'd_y', 'e_p', 'b/m', 'ntis'
]

y = df_filtered['ExRet']
X_lag = df_filtered[multi_predictors].shift(1)  # X_{t-1}

mv_sse_model = 0.0
mv_sse_bench = 0.0
mv_model_portfolio_rets = []
mv_bench_portfolio_rets = []

for t in range(initial_window, len(df_filtered)):
    # Expanding window uses data up to t-1
    y_train = y.iloc[:t]
    X_train = sm.add_constant(X_lag.iloc[:t])

    train = pd.concat([y_train, X_train], axis=1).dropna()
    y_tr = train['ExRet']
    X_tr = train.drop(columns=['ExRet'])

    mv_model = sm.OLS(y_tr, X_tr).fit()

    # Forecast r_t using X_{t-1} (which is X_lag at index t)
    x_prev = sm.add_constant(X_lag.iloc[[t]], has_constant="add")
    mv_forecast = float(mv_model.predict(x_prev).iloc[0])

    bench_forecast = y_tr.mean()
    realized = float(y.iloc[t])

    # OOS R² components
    mv_sse_model += (realized - mv_forecast) ** 2
    mv_sse_bench += (realized - bench_forecast) ** 2

    # Portfolio weights and returns (variance from expanding window)
    sigma_sq = y_tr.var(ddof=1)
    w_mv = np.clip((1/gamma) * (mv_forecast / sigma_sq), 0, 1.5)
    w_b = np.clip((1/gamma) * (bench_forecast / sigma_sq), 0, 1.5)

    mv_model_portfolio_rets.append(w_mv * realized)
    mv_bench_portfolio_rets.append(w_b * realized)

# Final multivariate OOS metrics
mv_oos_r2 = 1 - (mv_sse_model / mv_sse_bench)

ce_mv = np.mean(mv_model_portfolio_rets)- (gamma/2) * np.var(mv_model_portfolio_rets, ddof=1)
ce_b = np.mean(mv_bench_portfolio_rets) - (gamma/2) * np.var(mv_bench_portfolio_rets, ddof=1)
mv_delta_cev = (ce_mv - ce_b) * 12 * 100

print("Multivariate OLS (OOS) R2:", round(mv_oos_r2, 4))
print("Multivariate OLS Delta CEV:", round(mv_delta_cev, 2), "%")

Multivariate OLS (OOS) R2: -0.1195
Multivariate OLS Delta CEV: -0.7 %


After performing the multivariate OLS, we can compare the results with the in-sample and out-of-sample regressions.

In [9]:
# Calculating In-Sample Multivariate R-squared
y_is = df_filtered['ExRet'].iloc[1:] # Drop first row because it has no t-1
X_is = sm.add_constant(df_filtered[multi_predictors].shift(1).dropna())

mv_is_r2 = sm.OLS(y_is, X_is).fit().rsquared

# Add to your master table
r2_table.loc['MULTIVARIATE', :] = [mv_is_r2, mv_oos_r2, mv_delta_cev]

# Final formatting
r2_table_final = r2_table.copy()
r2_table_final['In-Sample R2'] = r2_table_final['In-Sample R2'].round(4)
r2_table_final['OOS R2'] = r2_table_final['OOS R2'].round(4)
r2_table_final['Delta CEV'] = r2_table_final['Delta CEV'].round(2).astype(str) + '%'
display(r2_table_final.sort_values(by='OOS R2', ascending=False))

,In-Sample R2,OOS R2,Delta CEV
Predictor,,,
tbl,0.0030,0.0021,-0.05%
infl_lagged,0.0014,0.0005,-0.02%
tms,0.0010,-0.0000,0.59%
lty,0.0020,-0.0026,-0.1%
dfr,0.0006,-0.0037,-0.01%
d_p,0.0024,-0.0039,-1.44%
ntis,0.0048,-0.0047,0.97%
d_e,0.0000,-0.0053,0.18%
ltr,0.0019,-0.0053,0.03%


From the multivariate regression, we get an in-sample R-squared of 0.0260, out-of-sample R-squared of -0.1195, and CEV of -0.7%. The results are displayed in our results table for clear comparison. 

The multivariate regression does well in the in-sample evaluation with an R-squared of 0.026. However, like many of the single predictors, it fails in the out-of-sample and the R-squared falls to -0.1195. The best single predictor model is tbl, as it has the highest out-of-sample R-squared of positive 0.0021. The multivariate model underperforms tbl in R-squared and also fails to achieve a positive CEV. Therefore, the multivariate model shows signs of overfitting and has weak predictive power.

## 3 Ridge and Lasso Regression

### 3.1 Regression

In [11]:
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.preprocessing import StandardScaler

# Define the candidate grid for Ridge
ridge_alphas = [0.001, 0.01, 0.1, 1, 10, 100]

# Re-include all 14 predictors
all_predictors = multi_predictors + ['d_e', 'tms']

# --- THE CRITICAL FIX: Explicitly Lag the Predictors ---
df_aligned = df_filtered.copy()
df_aligned[all_predictors] = df_aligned[all_predictors].shift(1)
df_aligned = df_aligned.dropna(subset=all_predictors) # Drop the first NaN row

# Storage for the OOS metrics
ridge_oos_results = {'sq_err': 0, 'port_rets': []}
lasso_oos_results = {'sq_err': 0, 'port_rets': []}
bench_sq_err = 0
bench_port_rets = []

# Loop uses the properly aligned dataframe
for t in range(initial_window, len(df_aligned)):
    # 1. Training block
    train_df = df_aligned.iloc[:t]
    y_train = train_df['ExRet']
    X_train = train_df[all_predictors]
    
    # 2. Forecasting block
    realized_t = df_aligned['ExRet'].iloc[t]
    x_prev_t = df_aligned[all_predictors].iloc[[t]] 
    
    # Scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_next_scaled = scaler.transform(x_prev_t)
    
    # --- REVERTED TO STANDARD CROSS-VALIDATION ---
    # Ridge uses default Leave-One-Out (cv=None)
    ridge_model = RidgeCV(alphas=ridge_alphas, cv=None).fit(X_train_scaled, y_train)
    ridge_forecast = ridge_model.predict(X_next_scaled)[0]
    
    # Lasso uses standard 5-Fold (cv=5)
    lasso_model = LassoCV(cv=5, max_iter=10000, tol=1e-3, selection='random').fit(X_train_scaled, y_train)
    lasso_forecast = lasso_model.predict(X_next_scaled)[0]
    
    # Benchmark and OOS R2 Components
    bench_forecast = y_train.mean()
    ridge_oos_results['sq_err'] += (realized_t - ridge_forecast)**2
    lasso_oos_results['sq_err'] += (realized_t - lasso_forecast)**2
    bench_sq_err += (realized_t - bench_forecast)**2
    
    # Portfolio Weights
    sigma_sq = y_train.var(ddof=1)
    
    w_ridge = np.clip((1 / gamma) * (ridge_forecast / sigma_sq), 0, 1.5)
    ridge_oos_results['port_rets'].append(w_ridge * realized_t)
    
    w_lasso = np.clip((1 / gamma) * (lasso_forecast / sigma_sq), 0, 1.5)
    lasso_oos_results['port_rets'].append(w_lasso * realized_t)
    
    w_bench = np.clip((1 / gamma) * (bench_forecast / sigma_sq), 0, 1.5)
    bench_port_rets.append(w_bench * realized_t)

    # Capture final lambdas for the report
    if t == len(df_aligned) - 1:
        final_ridge_lambda = ridge_model.alpha_
        final_lasso_lambda = lasso_model.alpha_

In [12]:
# Final R2 calculation
ridge_r2 = 1 - (ridge_oos_results['sq_err'] / bench_sq_err)
lasso_r2 = 1 - (lasso_oos_results['sq_err'] / bench_sq_err)

# Final CEV calculation function
def get_cev(rets, gamma=2.5):
    return np.mean(rets) - (gamma / 2) * np.var(rets, ddof=1)

ce_bench = get_cev(bench_port_rets)
ridge_delta_cev = (get_cev(ridge_oos_results['port_rets']) - ce_bench) * 12 * 100
lasso_delta_cev = (get_cev(lasso_oos_results['port_rets']) - ce_bench) * 12 * 100

# Create summary table
ridge_lasso_summary = {
    'Metric': [
        'Final Selected Lambda (λ)', 
        'Out-of-Sample R2', 
        'Annualized Delta CEV (%)'
    ],
    'Ridge Regression': [
        f"{final_ridge_lambda}", 
        f"{ridge_r2:.6f}", 
        f"{ridge_delta_cev:.2f}%"
    ],
    'Lasso Regression': [
        f"{final_lasso_lambda:.6f}", 
        f"{lasso_r2:.6f}", 
        f"{lasso_delta_cev:.2f}%"
    ]
}

problem_2a_df = pd.DataFrame(ridge_lasso_summary).set_index('Metric')

# Display the final table
problem_2a_df

,Ridge Regression,Lasso Regression
Metric,,
Final Selected Lambda (λ),100.0,0.003988
Out-of-Sample R2,-0.036379,-0.002447
Annualized Delta CEV (%),0.05%,-0.51%


### 3.2 Lasso Predictors

### Out-of-Sample Performance Comparison

## 4 Elastic Net and Summary

We will now use Elastic Net. Elastic Net combines Ridge regression and Lasso regression. Elastic Net estimates coefficients by minimizing the following formula:

$$
\sum_{t} (r_t - \alpha - x_{t-1}'\beta)^2 
+ \lambda \left[ 
\rho \sum_{j} |\beta_j| 
+ (1 - \rho) \sum_{j} \beta_j^2 
\right]
$$

$ \lambda $ controls the overall strength of regularization. $\rho \in [0,1] $ determines the mix between Lasso and Ridge, where $\rho = 1$ corresponds to Lasso, and $ \rho = 0 $ corresponds to Ridge.

### 4.1 Cross Validation

Now, we will perform cross validation with Elastic Net. We set possible values of $\rho$ as 0.1, 0.3, 0.5, 0.7, and 1.0.

In [24]:
# Elastic Net (CV) with expanding window, OOS R2 and ΔCEV

# Parameters
initial_window = 200
gamma = 2.5

# Use ALL 14 predictors for Elastic Net (including d_e and tms)
enet_predictors = [
    'd_e', 'svar', 'dfr', 'lty', 'ltr', 'infl_lagged',
    'tms', 'tbl', 'dfy', 'd_p', 'd_y', 'e_p', 'b/m', 'ntis'
]

# Targets and lagged predictors (so we predict r_t using X_{t-1})
y = df_filtered['ExRet']
X_lag = df_filtered[enet_predictors].shift(1)

# Storage
sse_model = 0.0
sse_bench = 0.0
model_portfolio_rets = []
bench_portfolio_rets = []

final_alpha = None
final_l1_ratio = None

# Time-series CV (inside each expanding window)
tscv = TimeSeriesSplit(n_splits=5)

for t in range(initial_window, len(df_filtered)):
    # Expanding window: train on months up to t-1
    y_train = y.iloc[:t]
    X_train = X_lag.iloc[:t]

    train = pd.concat([y_train, X_train], axis=1).dropna()
    y_tr = train['ExRet'].values
    X_tr = train[enet_predictors].values

    # Forecast row for month t uses X_{t-1} (which is X_lag at index t)
    x_prev = X_lag.iloc[[t]]
    if x_prev.isna().any(axis=1).iloc[0]:
        # If predictors missing at this date, skip this OOS month
        continue
    X_pred = x_prev[enet_predictors].values
    realized = float(y.iloc[t])

    # Elastic Net with standardization (important because predictors are on different scales)
    enet_pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("enet", ElasticNetCV(
            l1_ratio=[0.1, 0.3, 0.5, 0.7, 1.0],  # includes Lasso at 1.0
            alphas=30,
            cv=tscv,
            fit_intercept=True,
            max_iter=200000,
            random_state=0
        ))
    ])

    enet_pipe.fit(X_tr, y_tr)

    forecast = float(enet_pipe.predict(X_pred)[0])
    bench_forecast = float(np.mean(y_tr))

    # OOS R² components
    sse_model += (realized - forecast) ** 2
    sse_bench += (realized - bench_forecast) ** 2

    # Portfolio weights (variance from expanding window returns)
    sigma_sq = float(np.var(y_tr, ddof=1))
    w_model = np.clip((1/gamma) * (forecast / sigma_sq), 0, 1.5)
    w_bench = np.clip((1/gamma) * (bench_forecast / sigma_sq), 0, 1.5)

    model_portfolio_rets.append(w_model * realized)
    bench_portfolio_rets.append(w_bench * realized)

    # Save final-month selected hyperparameters (will end up as last loop's values)
    final_alpha = float(enet_pipe.named_steps["enet"].alpha_)
    final_l1_ratio = float(enet_pipe.named_steps["enet"].l1_ratio_)

    #Progress tracker
    if (t - initial_window) % 50 == 0:
        print(f"OOS step {t-initial_window} / {len(df_filtered)-initial_window}")


# Final OOS metrics
enet_oos_r2 = 1 - (sse_model / sse_bench)

ce_model = np.mean(model_portfolio_rets) - (gamma/2) * np.var(model_portfolio_rets, ddof=1)
ce_bench = np.mean(bench_portfolio_rets) - (gamma/2) * np.var(bench_portfolio_rets, ddof=1)
enet_delta_cev = (ce_model - ce_bench) * 12 * 100

print("Elastic Net (final CV) alpha (λ):", final_alpha)
print("Elastic Net (final CV) l1_ratio (ρ):", final_l1_ratio)
print("Elastic Net OOS R2:", round(enet_oos_r2, 4))
print("Elastic Net Delta CEV:", round(enet_delta_cev, 2), "%")


OOS step 0 / 977
OOS step 50 / 977
OOS step 100 / 977
OOS step 150 / 977
OOS step 200 / 977
OOS step 250 / 977
OOS step 300 / 977
OOS step 350 / 977
OOS step 400 / 977
OOS step 450 / 977
OOS step 500 / 977
OOS step 550 / 977
OOS step 600 / 977
OOS step 650 / 977
OOS step 700 / 977
OOS step 750 / 977
OOS step 800 / 977
OOS step 850 / 977
OOS step 900 / 977
OOS step 950 / 977
Elastic Net (final CV) alpha (λ): 0.03987559111008384
Elastic Net (final CV) l1_ratio (ρ): 0.1
Elastic Net OOS R2: 0.0002
Elastic Net Delta CEV: 0.02 %


From the Elastic Net cross validation, we select the $\rho$ value of 0.1, and $\lambda$ value of 0.0399. This means that the model behaves a lot more like Ridge regression compared to Lasso.
 
We get that the out-of-sample R-squared is 0.0002, and the delta CEV is 0.02%. This means that there are only tiny gains with the Elastic Net. We will go on to compare these results with the other models.

### 4.2 Summarize Findings

### 4.3 Conclusion

## 5.0 Appendix: Note on AI Usage
Generative AI was used to complete this project. Specifically, OpenAI's GPT 5.2 was used to complete coding tasks. The process I used was to ask the tool to complete code on a certain part of the question. For example, a prompt would be "help me parse the date column to convert it to datetime format". For markdown, AI was only used to generate mathematical formulas. AI was not used to generate blocks of markdown. All the additional writing and explanations was done in our own words.  
 
Overall, AI assistance was helpful for this lab report. It helped to solve issues we had working with Git, as well as debugging when things went wrong. For example, when Elastic Net's runtime was too long, it helped us know how to modify code to reduce its runtime.